In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('full_model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=25, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=25, out_features=5, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([25, 36])
l1.bias torch.Size([25])
l2.weight torch.Size([5, 25])
l2.bias torch.Size([5])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {0.08789841830730438, -0.9939795136451721, 0.8244245648384094, -0.2078191637992859, -0.13660389184951782, -0.21693475544452667, -0.004347869660705328, 0.9468597173690796, 0.10515663772821426, 0.1689077466726303, 0.015779733657836914, 1.7727829217910767, 0.13283146917819977, -0.08736374974250793, -0.12113751471042633, -3.6775963306427, 0.1586243361234665, 0.014412656426429749, 0.08353310823440552, -0.08219151943922043, -0.030531112104654312, 0.1260993927717209, -0.043212175369262695, 0.14358770847320557, -0.03386224806308746};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {0.37281033396720886, -1.9228103160858154, -3.9149246215820312, -0.22645704448223114, 3.4774186611175537};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{-0.10837434977293015, -0.042834214866161346, -0.06728044152259827, 0.10916683822870255, 0.035062581300735474, -0.059086594730615616, -0.1523810774087906, 0.12333311140537262, 0.005926420446485281, 0.14273592829704285, 0.10246676951646805, -0.13944530487060547, -0.08624669909477234, 0.10305309295654297, 0.10801130533218384, -0.11679887771606445, -0.1329302042722702, -0.09388552606105804, 0.12336407601833344, 0.013888083398342133, 0.16502320766448975, -0.06333436071872711, 0.06897539645433426, 0.06798169761896133, -0.20151649415493011, 0.12333597987890244, 0.061296962201595306, 0.10252537578344345, 0.07178738713264465, -0.12900876998901367, -0.19626827538013458, 0.005243740975856781, -0.0749170109629631, -0.04733368009328842, 0.06752575188875198, -0.11684709042310715},
{-2.0867204666137695, 1.4256112575531006, 1.0425055027008057, -0.8561283946037292, 0.4456169009208679, -0.36046987771987915, -3.4837710857391357, 4.411922454833984, -1.27

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{0.23916663229465485, 0.23125231266021729, 0.1300157904624939, -0.050731223076581955, -0.16370391845703125, -0.03588620573282242, 0.22558005154132843, 0.0792354866862297, 0.105376236140728, 0.06998691707849503, -0.0836247131228447, 0.2834746241569519, 0.1048162505030632, -0.04826853796839714, 0.15671268105506897, -1.020902395248413, 0.11692745983600616, 0.07308211922645569, -0.08521554619073868, -0.23254694044589996, 0.1368531882762909, 0.16276036202907562, -0.09912369400262833, -0.08722428977489471, -0.03132258355617523},
{0.0777340978384018, -0.20522862672805786, -0.016094082966446877, -0.12519219517707825, 0.18423636257648468, 0.017589712515473366, 0.033154476433992386, -0.6912493705749512, 0.239850714802742, 0.078799769282341, 0.10015853494405746, -0.10354243963956833, -0.036039941012859344, -0.04315580800175667, 0.017911270260810852, 0.3249325156211853, -0.006432145833969116, -0.1470727175474167, -0.01314866729080677, -0.05621649

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [13]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [14]:
data = [torch.Tensor(X_test.iloc[x].astype(float)) for x in range(5)]

In [15]:
data

[tensor([ 1.0200e+00, -9.7000e-01,  3.9000e-02,  4.0791e-01,  3.8750e-01,
          2.8865e-01,  1.5900e+00, -3.6000e-01,  5.8650e-01,  5.6652e-01,
          9.5000e-01,  5.1847e-01, -4.0000e-02, -1.8100e+00, -4.7725e-01,
          4.9297e-01,  8.3750e-01,  4.3256e-01,  7.0710e+01, -2.4950e+02,
         -2.2441e+01,  6.3077e+01,  2.7077e+01,  4.3705e+01,  2.7337e+02,
         -2.6333e+02,  7.9735e+00,  1.1816e+02,  5.1778e+01,  7.5204e+01,
          1.8905e+02, -2.6107e+02, -2.5219e+01,  9.7192e+01,  5.0713e+01,
          6.7856e+01]),
 tensor([ 2.7000e-01, -2.0000e-01,  1.3750e-02,  1.2247e-01,  2.2250e-01,
          1.0844e-01,  3.5000e-01, -2.7000e-01,  7.3750e-02,  1.5127e-01,
          1.8750e-01,  1.2025e-01,  1.7000e-01, -2.8000e-01, -6.5250e-02,
          1.1041e-01,  1.7250e-01,  9.2250e-02,  4.9850e+01, -5.8070e+01,
         -1.9844e+01,  2.5640e+01,  3.5660e+01,  2.0699e+01,  1.1155e+02,
         -1.2077e+02, -4.3960e+01,  5.7641e+01,  7.7215e+01,  4.6109e+01,
          1.04

In [16]:
with torch.no_grad():
    for d in data:
        start_time = time.time()
        output = model(d)
        action = np.argmax(output)
#         print(f"Inference time: {time.time() - start_time}")    
        print(action)

tensor(3)
tensor(0)
tensor(4)
tensor(0)
tensor(0)


In [17]:
for x in range(5):
    print(y_test[x])

3
0
4
0
4


## Compare software-hardware running time and accuracy

In [18]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 3; Ground truth value: 3
Inference time: 3.0169 ms


In [19]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 92.50%
Inference time: 0.0379 s


In [20]:
len(X_test)

80